In [1]:
import pandas as pd
import os
import statsmodels.api as sm
from sklearn import linear_model
import numpy as np

## This notebooks gathers and cleans dataframes for vehicle registrations, population, GDP, GDP by industry, C02 emissions and energy consumption. 

### The goal is to get all these dataframes in the same format, so we can use them to create a multiple linear regression model. Once we have all the dataframes cleaned and in the correct format, we will write them out, so that the cleaned dataframes can be used for analysis. 



### Data Gathering
#### This section of the notebooks reads in the data files and stores them im pandas dataframes.
The dataframes frames in this section all have columns of represting years ranging from [1967-2020] and rows for each state. 

In [ ]:
csv_path = os.path.join(os.getcwd(), "Data/uncleaned/csv")
excel_path = os.path.join(os.getcwd(), "Data/uncleaned/excel")

In [ ]:
#Read in all datasets here 

vehicle_registration_df = pd.read_csv(os.path.join(csv_path, "vehicle_registrations_by_state.csv"))
energy_consumption_per_real_gdp_df = pd.read_csv(os.path.join(csv_path, "energy_consumption_per_real_gdp.csv"))
current_dollar_gdp_df = pd.read_csv(os.path.join(csv_path, "Current_dollar_GDP.csv")) #in millions
total_consuption_df = pd.read_csv(os.path.join(csv_path, "total_consuption.csv")) #in billion Btu
industy_gdp_by_state_df = pd.read_csv(os.path.join(csv_path, "industy_gdp_by_state.csv"))
total_population_df = pd.read_csv(os.path.join(csv_path, "total_population.csv"))
real_gdp_df = pd.read_csv(os.path.join(csv_path, "real_GDP.csv")) #in millions
ghg_emissions_df = pd.read_csv(os.path.join(csv_path, "GHG_Emissions.csv"))
co2_emissions_df = pd.read_excel(os.path.join(excel_path, "co2_emissions.xlsx"))

In [2]:
state_names=["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
state_abbreviations = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [3]:
unnamed_columns_to_drop = ['Unnamed: 91', 'Unnamed: 92', 'Unnamed: 93', 'Unnamed: 94',
       'Unnamed: 95', 'Unnamed: 96', 'Unnamed: 97', 'Unnamed: 98',
       'Unnamed: 99', 'Unnamed: 100','Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64', 'Unnamed: 65',
       'Unnamed: 66', 'Unnamed: 67', 'Unnamed: 68', 'Unnamed: 69',
       'Unnamed: 70', 'Unnamed: 71', 'Unnamed: 72', 'Unnamed: 73',
       'Unnamed: 74', 'Unnamed: 75', 'Unnamed: 76', 'Unnamed: 77',
       'Unnamed: 78', 'Unnamed: 79', 'Unnamed: 80', 'Unnamed: 81',
       'Unnamed: 82', 'Unnamed: 83', 'Unnamed: 84', 'Unnamed: 85',
       'Unnamed: 86', 'Unnamed: 87', 'Unnamed: 88', 'Unnamed: 89',
       'Unnamed: 90', 'Unnamed: 61', 'Unnamed: 0']

In [4]:
def clean_dataframe(df):
    srings_to_replace = ["(NA)", "(L)", "(D)", "Change", "(2000–2018)", "Percent", "Absolute"]
    unnamed_to_drop = list(set(df.columns).intersection(unnamed_columns_to_drop))
    df = df.drop(columns = unnamed_to_drop)
    
    null_values_allowed_before_column_is_dropped = 40
    columns_to_drop = []
    first_col = df.columns[0]
    for index, row in df.iterrows():
        df.at[index, first_col] = str(df.at[index, first_col]).replace("(Items)", "").strip()

    for col in df.columns[1:]:
        df.loc[df[col].isin(srings_to_replace), col] = np.nan
        df[col] = df[col].astype(float)
        
        if(df[col].isna().sum() > null_values_allowed_before_column_is_dropped):
            columns_to_drop.append(col)
        else:
            df[col].fillna(value=pd.to_numeric(df[col], errors='coerce').mean(), inplace=True)
    df = df.drop( columns = columns_to_drop)
    return df

In [5]:
csv_path = os.path.join(os.getcwd(), "Data/cleaned/csv")
excel_path = os.path.join(os.getcwd(), "Data/cleaned/excel")

In [10]:
#read in and clean all the NOA files
files = os.listdir(os.path.join(os.getcwd(), "Data/uncleaned/csv/NOA"))
for file in files:
    if(".csv" in file):
        print(os.path.join(csv_path,'NOA', file))
        df = pd.read_csv(os.path.join(os.getcwd(), "Data/uncleaned/csv/NOA", file), index_col=None)
        df = clean_dataframe(df)
        df.to_csv(os.path.join(csv_path,'NOA', file))
    

/Users/haleyhartin/Projects/State-Of-The-Energy/Data/cleaned/csv/NOA/DYTS.csv
/Users/haleyhartin/Projects/State-Of-The-Energy/Data/cleaned/csv/NOA/HTDD.csv
/Users/haleyhartin/Projects/State-Of-The-Energy/Data/cleaned/csv/NOA/TMAX.csv
/Users/haleyhartin/Projects/State-Of-The-Energy/Data/cleaned/csv/NOA/HX01.csv
/Users/haleyhartin/Projects/State-Of-The-Energy/Data/cleaned/csv/NOA/DSNW.csv
/Users/haleyhartin/Projects/State-Of-The-Energy/Data/cleaned/csv/NOA/DT32.csv
/Users/haleyhartin/Projects/State-Of-The-Energy/Data/cleaned/csv/NOA/EMXT.csv
/Users/haleyhartin/Projects/State-Of-The-Energy/Data/cleaned/csv/NOA/EMXP.csv
/Users/haleyhartin/Projects/State-Of-The-Energy/Data/cleaned/csv/NOA/DSND.csv
/Users/haleyhartin/Projects/State-Of-The-Energy/Data/cleaned/csv/NOA/EVAP.csv
/Users/haleyhartin/Projects/State-Of-The-Energy/Data/cleaned/csv/NOA/PSUN.csv
/Users/haleyhartin/Projects/State-Of-The-Energy/Data/cleaned/csv/NOA/DP1X.csv
/Users/haleyhartin/Projects/State-Of-The-Energy/Data/cleaned/csv

In [ ]:
vehicle_registration_df.drop(index = [0,1,2,3,4,5,6,7,8,9,61,62,64,63], inplace = True)
vehicle_registration_df = clean_dataframe(vehicle_registration_df)
vehicle_registration_df =  vehicle_registration_df[vehicle_registration_df['Years'].isin(state_names)]
vehicle_registration_columns = vehicle_registration_df.columns

In [ ]:
total_population_df = clean_dataframe(total_population_df)
total_population_df =  total_population_df[total_population_df['State'].isin(state_abbreviations)]
total_population_columns = total_population_df.columns

In [ ]:
total_consuption_df = clean_dataframe(total_consuption_df)
total_consuption_df =  total_consuption_df[total_consuption_df['State'].isin(state_abbreviations)]
total_consuption_df_columns = total_consuption_df.columns

In [ ]:
real_gdp_df = clean_dataframe(real_gdp_df)
real_gdp_df =  real_gdp_df[real_gdp_df['State'].isin(state_abbreviations)]
real_gdp_df_columns = real_gdp_df.columns

In [ ]:
industy_gdp_by_state_df = industy_gdp_by_state_df.drop(columns = ['GeoFIPS','Region','TableName','LineCode','IndustryClassification','Description','Unit'])
industy_gdp_by_state_df = clean_dataframe(industy_gdp_by_state_df)
industy_gdp_by_state_df = industy_gdp_by_state_df.groupby('GeoName').mean().reset_index()
industy_gdp_by_state_df =  industy_gdp_by_state_df[industy_gdp_by_state_df['GeoName'].isin(state_names)]
industy_gdp_by_state_df_columns = industy_gdp_by_state_df.columns

In [ ]:
co2_emissions_df = co2_emissions_df.rename({'Table 2. State energy-related carbon dioxide emissions by year, adjusted (2000–2018)': 'State', 'Unnamed: 1': '2000', 'Unnamed: 2': '2001', 'Unnamed: 3': '2002', 'Unnamed: 4': '2003', 'Unnamed: 5': '2004',
       'Unnamed: 6': '2005', 'Unnamed: 7': '2006', 'Unnamed: 8': '2007', 'Unnamed: 9': '2008', 'Unnamed: 10': '2009',
       'Unnamed: 11': '2010', 'Unnamed: 12': '2011', 'Unnamed: 13': '2012', 'Unnamed: 14': '2013',
       'Unnamed: 15': '2014', 'Unnamed: 16': '2015', 'Unnamed: 17': '2016', 'Unnamed: 18': '2017',
       'Unnamed: 19': '2018', 'Unnamed: 20': '2019', 'Unnamed: 21': '2020'}, axis =1).reset_index()
co2_emissions_df = co2_emissions_df.drop(columns = ['index'])
co2_emissions_df =  co2_emissions_df[co2_emissions_df['State'].isin(state_names)]
co2_emissions_df_columns = co2_emissions_df.columns

In [ ]:
# Divide GDP and industry GDP by cooresponding population values to normalize population for the regression
columns_to_evaluate = list(set(real_gdp_df.columns).intersection(industy_gdp_by_state_df.columns).intersection(total_population_df.columns))

for col in columns_to_evaluate:
    for i in range(0,50):
        population = total_population_df.iloc[i][col]
        if(population > 0):
            real_gdp_df.iloc[i][col] = real_gdp_df.iloc[i][col] / population
            industy_gdp_by_state_df.iloc[i][col] = industy_gdp_by_state_df.iloc[i][col] / population


### Write the data out to new files
#### This section of the notebooks writes the cleaned dataframes out into new files that can be used by other notebooks


In [ ]:
vehicle_registration_df.to_csv(os.path.join(csv_path, "vehicle_registrations_by_state.csv"))
energy_consumption_per_real_gdp_df.to_csv(os.path.join(csv_path, "energy_consumption_per_real_gdp.csv"))
current_dollar_gdp_df.to_csv(os.path.join(csv_path, "Current_dollar_GDP.csv")) #in millions
total_consuption_df.to_csv(os.path.join(csv_path, "total_consuption.csv")) #in billion Btu
industy_gdp_by_state_df.to_csv(os.path.join(csv_path, "industy_gdp_by_state.csv"))
total_population_df.to_csv(os.path.join(csv_path, "total_population.csv"))
real_gdp_df.to_csv(os.path.join(csv_path, "real_GDP.csv")) #in millions
co2_emissions_df.to_excel(os.path.join(excel_path, "co2_emissions.xlsx"))